In [11]:
# ==============================================================
# 📘 Modelo: Clasificación de Tipo de Semana (Recupero / Mantener / Carga)
# ==============================================================
# Este notebook entrena un modelo de clasificación supervisada para predecir
# el tipo de semana del jugador según su carga actual y la semana anterior.
# ==============================================================

import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import os

# --------------------------------------------------------------
# 🔹 1. Cargar datos desde el Data Warehouse
# --------------------------------------------------------------

# Ruta al DW
db_path = "C:/Users/Nico/Desktop/DATA SCIENCE/PP- VOLUNTAREADO/chivas-ml/data/external/chivas_dw.sqlite"
conn = sqlite3.connect(db_path)

query = """
SELECT * FROM vw_tipo_semana_modelo_extendida_v3
"""

df = pd.read_sql(query, conn)
conn.close()
print(f"Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")



Datos cargados: 744 filas, 25 columnas


In [12]:
# =============================================
# 2.1 FILTRO DE JUGADORES Y MICRO-CICLOS
# =============================================

print(f"Registros cargados pre filtrado: {len(df)}\n")

# Jugadores que deben excluirse
jugadores_excluir = [1, 2, 3, 12, 30]

# Filtrar jugadores no válidos
df = df[~df["id_jugador"].isin(jugadores_excluir)]


microciclos_excluir = [1, 2, 3, 4, 5, 6, 7]

# Filtrar jugadores no válidos
df = df[~df["microciclo_actual"].isin(microciclos_excluir)]

print(f"Registros cargados post filtrado: {len(df)}\n")
# Distribución de la etiqueta
print("\nDistribución de tipo_semana_next:")
print(df['tipo_semana_next'].value_counts(normalize=True) * 100)


Registros cargados pre filtrado: 744

Registros cargados post filtrado: 442


Distribución de tipo_semana_next:
tipo_semana_next
RECUPERO    40.144231
CARGA       37.019231
MANTENER    22.836538
Name: proportion, dtype: float64


In [13]:
# 3️⃣ LIMPIEZA Y SELECCIÓN DE VARIABLES
# Eliminamos columnas no necesarias o identificadores
drop_cols = [
    'id_jugador', 'nombre_jugador', 'Microciclo_Num', 
    'Fecha_Inicio', 'Fecha_Fin'
]
df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')

# Eliminamos filas con nulos críticos
df = df.dropna(subset=['CT_total_actual', 'CE_total_actual', 'CS_total_actual', 'CR_total_actual'])

print(f"✅ Filas tras limpieza: {len(df)}")



✅ Filas tras limpieza: 442


In [14]:
condiciones = [
    (df['CT_total_next'] < df['CT_total_actual'] * 0.9),
    (df['CT_total_next'].between(df['CT_total_actual'] * 0.9, df['CT_total_actual'] * 1.1)),
    (df['CT_total_next'] > df['CT_total_actual'] * 1.1)
]
valores = ['RECUPERO', 'MANTENER', 'CARGA']
df['target'] = np.select(condiciones, valores, default='MANTENER')

print(df['target'].value_counts())

target
RECUPERO    167
CARGA       154
MANTENER    121
Name: count, dtype: int64


In [15]:
# 5️⃣ SELECCIÓN DE FEATURES
FEATURES = [
    # Cargas actuales
    'CT_total_actual', 'CE_total_actual', 'CS_total_actual', 'CR_total_actual',

    # Cargas de la semana siguiente
    'CT_total_next', 'CE_total_next', 'CS_total_next', 'CR_total_next',

    # Planificación semanal siguiente
    'entrenos_total_next', 'descansos_total_next', 'partidos_total_next',
    'entrenos_pre_partido_next', 'entrenos_post_partido_next', 'descansos_pre_partido_next'
] + [c for c in df.columns if c.startswith('Pos_') or c.startswith('Lin_')]

X = df[FEATURES].copy()
y = df['target']

# 6️⃣ SPLIT Y ESCALADO
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 7️⃣ ENTRENAMIENTO
model = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    random_state=42,
    class_weight='balanced_subsample'
)
model.fit(X_train_scaled, y_train)

# 8️⃣ EVALUACIÓN
y_pred = model.predict(X_test_scaled)
print("📊 Reporte de Clasificación:\n", classification_report(y_test, y_pred))
print("📈 Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))

# 9️⃣ CROSS-VALIDATION
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='f1_macro')
print(f"✅ F1 Macro (CV 5 folds): {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

# 🔟 EXPORTAR MODELO Y SCALER
output_dir = r"C:\Users\Nico\Desktop\DATA SCIENCE\PP- VOLUNTAREADO\chivas-ml\src\chivas_ml\ml\registry\Prueba_ML2.0\Modelo_Tipo_Semana"
os.makedirs(output_dir, exist_ok=True)

joblib.dump(model, os.path.join(output_dir, "model_tipo_semana_rf.pkl"))
joblib.dump(scaler, os.path.join(output_dir, "scaler_tipo_semana.pkl"))

print(f"💾 Modelo y scaler guardados en: {output_dir}")




📊 Reporte de Clasificación:
               precision    recall  f1-score   support

       CARGA       1.00      0.97      0.98        31
    MANTENER       0.90      0.75      0.82        24
    RECUPERO       0.85      0.97      0.90        34

    accuracy                           0.91        89
   macro avg       0.92      0.90      0.90        89
weighted avg       0.91      0.91      0.91        89

📈 Matriz de Confusión:
 [[30  1  0]
 [ 0 18  6]
 [ 0  1 33]]
✅ F1 Macro (CV 5 folds): 0.912 ± 0.017
💾 Modelo y scaler guardados en: C:\Users\Nico\Desktop\DATA SCIENCE\PP- VOLUNTAREADO\chivas-ml\src\chivas_ml\ml\registry\Prueba_ML2.0\Modelo_Tipo_Semana


### 🧠 Informe técnico – Modelo de Clasificación del Tipo de Semana (Recupero / Mantener / Carga)
### 🎯 Objetivo

Redefinir el modelo de clasificación semanal para representar de forma más fisiológica y coherente las transiciones de carga entre microciclos, reemplazando el esquema anterior (baja / media / alta) por tres categorías basadas en la evolución real de las cargas semanales:

RECUPERO: semana de reducción significativa de carga.

MANTENER: semana de estabilidad en volumen e intensidad.

CARGA: semana de incremento de carga planificado o progresivo.

### 🧩 Estructura de datos

El modelo se entrena a partir de la vista vw_tipo_semana_modelo_extendida_v2, que integra:

Totales actuales y siguientes:

CT_total_actual, CE_total_actual, CS_total_actual, CR_total_actual

CT_total_next, CE_total_next, CS_total_next, CR_total_next

Planificación futura:

entrenos_total_next, descansos_total_next, partidos_total_next

entrenos_pre_partido_next, entrenos_post_partido_next, descansos_pre_partido_next

Codificación posicional (Pos_, Lin_)

### 🧮 Generación del Target

El target se construyó comparando la variación relativa entre cargas totales de semanas consecutivas:

condiciones = [
    (df['CT_total_next'] < df['CT_total_actual'] * 0.9),
    (df['CT_total_next'].between(df['CT_total_actual'] * 0.9, df['CT_total_actual'] * 1.1)),
    (df['CT_total_next'] > df['CT_total_actual'] * 1.1)
]
valores = ['RECUPERO', 'MANTENER', 'CARGA']


Esto define el tipo de semana en función del comportamiento fisiológico esperado:

Relación entre semanas	Clasificación	Interpretación

↓ > 10 %	RECUPERO	Semana de descarga o regeneración

±10 %	MANTENER	Continuidad en carga o estabilización

↑ > 10 %	CARGA	Semana de estímulo progresivo

⚙️ Variables del modelo

#### Features utilizados:

Cargas semanales actuales y siguientes (CT, CE, CS, CR)

Variables de planificación futura (entrenos_total_next, descansos_total_next, etc.)

Variables posicionales (Pos_, Lin_)

Modelo: Random Forest Classifier
Hiperparámetros:

n_estimators=300

max_depth=12

class_weight='balanced_subsample'

### 📊 Resultados

| Clase                     | Precisión | Recall | F1-Score          | Soporte |
| ------------------------- | --------- | ------ | ----------------- | ------- |
| **CARGA**                 | 0.91      | 0.87   | 0.89              | 23      |
| **MANTENER**              | 0.83      | 0.83   | 0.83              | 23      |
| **RECUPERO**              | 0.95      | 0.97   | 0.96              | 38      |
| **Accuracy**              | –         | –      | **0.90**          | 84      |
| **F1 Macro (CV 5 folds)** | –         | –      | **0.905 ± 0.042** | –       |


### 📈 Matriz de confusión:

| Real \ Predicha | **CARGA** | **MANTENER** | **RECUPERO** |
| --------------- | --------- | ------------ | ------------ |
| **CARGA**       | 20        | 3            | 0            |
| **MANTENER**    | 2         | 19           | 2            |
| **RECUPERO**    | 0         | 1            | 37           |


### 🧠 Interpretación

Coherencia fisiológica lograda:
El modelo distingue claramente entre semanas de carga, mantenimiento y recuperación.

Baja confusión entre extremos (RECUPERO ↔ CARGA):
Refleja una correcta comprensión del comportamiento de variación semanal.

MANTENER muestra leve solapamiento con las otras clases, algo esperable dada su naturaleza intermedia.

Excelente estabilidad del modelo:
Desvío estándar de solo ±0.04 en F1 macro durante validación cruzada.

### 🔗 Conclusiones

✅ El nuevo modelo sustituye exitosamente al esquema “baja / media / alta”.

🧩 Permite un control más realista de las cargas, considerando el comportamiento real entre semanas consecutivas.

📈 Su precisión y robustez lo vuelven adecuado para integrarse en el flujo jerárquico de predicción (predict_pipeline), aportando una capa más interpretable al sistema.

🚀 Puede ser extendido para entrenar modelos semanales de Distancia Total, Carga Explosiva y Carga Sostenida, usando como input su predicción (tipo_semana_pred_v2).